In [6]:
!pip install langchain langchain-community
!pip install sentence-transformers
!pip install chromadb
!pip install transformers
!pip install torch torchvision torchaudio
!pip install huggingface_hub
!pip install pypdf
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 2

In [4]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

# Login to Hugging Face
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login
import os

In [8]:
data_load = PyPDFLoader("/kaggle/input/breast-cancer/Breast Cancer1.pdf")
data = data_load.load()
data[0]

Document(metadata={'source': '/kaggle/input/breast-cancer/Breast Cancer1.pdf', 'page': 0}, page_content='1   \n \nAbout Breast Cancer  \nGet basic  information  about  breast  cancer,  such  as the different  types,  where  they start, \nimportant statistics, and current research topics.  \n \nBreast Cancer Basics  \n \nGet an overview of the different types of breast cancer and where they start.  \n \n● What Is Breast Cancer?  \n● What Causes Breast Cancer?  \n \nTypes of Breast Cancer  \n \nThere  are several  types  of breast  cancer.  The type of breast  cancer  you have  depends \non where in the breast it started and other factors.  \n \n● Types of Breast Cancer Overview  \n● Ductal Carcinoma in Situ (DCIS)  \n● Invasive Breast Cancer (IDC/ILC)  \n● Triple -negative Breast Cancer  \n● Angiosarcoma of the Breast  \n● Inflammatory Breast Cancer  \n● Paget Disease o f the Breast  \n● Phyllodes Tumors  \n \nResearch and Statistics  \n \nSee the latest  estimates  for new cases  and d

In [9]:
print(data[0].page_content)

1   
 
About Breast Cancer  
Get basic  information  about  breast  cancer,  such  as the different  types,  where  they start, 
important statistics, and current research topics.  
 
Breast Cancer Basics  
 
Get an overview of the different types of breast cancer and where they start.  
 
● What Is Breast Cancer?  
● What Causes Breast Cancer?  
 
Types of Breast Cancer  
 
There  are several  types  of breast  cancer.  The type of breast  cancer  you have  depends 
on where in the breast it started and other factors.  
 
● Types of Breast Cancer Overview  
● Ductal Carcinoma in Situ (DCIS)  
● Invasive Breast Cancer (IDC/ILC)  
● Triple -negative Breast Cancer  
● Angiosarcoma of the Breast  
● Inflammatory Breast Cancer  
● Paget Disease o f the Breast  
● Phyllodes Tumors  
 
Research and Statistics  
 
See the latest  estimates  for new cases  and deaths  of breast  cancer  in the US and what 
research is being done.  



In [10]:
page_contents = [doc.page_content for doc in data]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(data)
len(documents)

1105

In [11]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

pdf_embedding = embedder.encode([doc.page_content for doc in documents])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [12]:
#indexing process:
client = chromadb.Client()

# Create a collection for PDF document embeddings
pdf_collection = client.create_collection("pdf_documentss")

# Generate unique IDs for each document
ids = [str(i) for i in range(len(documents))]

# Add the PDF text and its embedding to Chroma collection
# Use the content of each Document object and include the generated IDs
pdf_collection.add(
    ids=ids,  # Unique IDs for each document
    documents=[doc.page_content for doc in documents],  # Store the raw text
    embeddings=pdf_embedding  # Store the corresponding embedding
)

In [13]:
def query_chroma(query):
    # Generate embedding for the user query
    query_embedding = embedder.encode([query])

    # Retrieve relevant documents from Chroma
    results = pdf_collection.query(
        query_embeddings=query_embedding,
        n_results=5  # Get top 5 relevant documents
    )
    retrieved_docs = results['documents']
    return retrieved_docs

Text generation

reduce
 fine tuning best library
 Quantization

In [14]:
model_id = 'meta-llama/Llama-3.1-8B-Instruct' 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [302]:
query = "who has more goals cristiano ronaldo or lionel messi?"
retrieved_docs = query_chroma(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [303]:
retrieved_docs

[['What is a 5 -year relative survival rate?  \n \nA relative survival rate compares people with the same type and stage of cancer to \npeople in the overall population. For example, if the 5-year relative survival rate for a \nspecific  stage  of breast  cancer  in men is 80%,  it means  that men who have  that cancer  \nare, on average, about 80% as likely as men who don’t have that cancer to live for at \nleast 5 years after being diagnosed.  \n \nWhere do these numbers come from?  \n \nThe American Cancer Society relies on information from the Surveillance, \nEpidemiology,  and End Results  (SEER)  database,  maintained  by the National  Cancer \nInstitute (NCI), to provide survival statistics for different types of cancer.  \n \nThe SEER database tracks 5 -year relative survival rates for breast cancer in men in the',
  'levels.  \n \nAging  \nAging  is an important  risk factor  for the development  of breast  cancer  in men.  The risk \nof breast cancer goes up as a man ages. On

Prompt Engineering

In [304]:
def format_chat_template(question, retrieved_context):
    # Template to answer the question with strict relevance
    chat_template = f"""
You are an intelligent assistant. You can only use the following context to answer the question. If the question is not related to the context or ypu don't understand it, respond with "i do not know."
dont say any notes based on your data.
dont say any informations or data that is not related to the context.
if the content in the question is not related to the context dont say anything or do not try to prove anything just say i do not know.
do not say any additional information by yourself if it is not relevant to the following context provided.
Question: {question}

Context: {retrieved_context}

Answer:
"""
    return chat_template

In [305]:
def extract_final_answer(response):
    # Expecting the response to be of the format: "Reasoning: ... Final Answer: <actual_answer>"
    final_answer_prefix = "Final Answer:"
    final_answer_prefix2 = "Answer:"
    if final_answer_prefix in response:
        # Extract the answer part only
        answer = response.split(final_answer_prefix)[-1].strip()
        # Optionally, stop at the first period or any other stop marker
        answer = answer.split('.')[0] + '.' if '.' in answer else answer
        return answer.strip()
    elif final_answer_prefix2 in response:
        answer = response.split(final_answer_prefix2)[-1].strip()
        # Optionally, stop at the first period or any other stop marker
        answer = answer.split('.')[0] + '.' if '.' in answer else answer
        return answer.strip()
    
    return response.strip()  # Fallback if format isn't matched

In [306]:
# Join retrieved documents into a single context
retrieved_context = " ".join(retrieved_docs[0])

# Format the input using the chat template
formatted_input = format_chat_template(query, retrieved_context)

formatted_input

'\nYou are an intelligent assistant. You can only use the following context to answer the question. If the question is not related to the context or ypu don\'t understand it, respond with "i do not know."\ndont say any notes based on your data.\ndont say any informations or data that is not related to the context.\nif the content in the question is not related to the context dont say anything or do not try to prove anything just say i do not know.\ndo not say any additional information by yourself if it is not relevant to the following context provided.\nQuestion: who has more goals cristiano ronaldo or lionel messi?\n\nContext: What is a 5 -year relative survival rate?  \n \nA relative survival rate compares people with the same type and stage of cancer to \npeople in the overall population. For example, if the 5-year relative survival rate for a \nspecific  stage  of breast  cancer  in men is 80%,  it means  that men who have  that cancer  \nare, on average, about 80% as likely as me

In [307]:
inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

In [308]:

output = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id,no_repeat_ngram_size=2,repetition_penalty=1.5,early_stopping=True )

# Decode the response
response = tokenizer.decode(output[0], skip_special_tokens=True)

final_answer = response.split("Answer:")[-1].strip()
    
if "i do not know" in final_answer.lower():
    final_answer = "I don't know."

print(final_answer)

I don't know.


In [309]:
print(response)


You are an intelligent assistant. You can only use the following context to answer the question. If the question is not related to the context or ypu don't understand it, respond with "i do not know."
dont say any notes based on your data.
dont say any informations or data that is not related to the context.
if the content in the question is not related to the context dont say anything or do not try to prove anything just say i do not know.
do not say any additional information by yourself if it is not relevant to the following context provided.
Question: who has more goals cristiano ronaldo or lionel messi?

Context: What is a 5 -year relative survival rate?  
 
A relative survival rate compares people with the same type and stage of cancer to 
people in the overall population. For example, if the 5-year relative survival rate for a 
specific  stage  of breast  cancer  in men is 80%,  it means  that men who have  that cancer  
are, on average, about 80% as likely as men who don’t hav